### FewShotPromptTemplate and Fewshot Learning
- Fewshot means you are going to give an example to models to lead better answer.
- Instead of delivering prompt every single time, it is more efficient to give an example to model what I want.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variable=["country"],
# )
# t.format(country="France")


t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country="France") 


'What is the capital of France'

In [2]:
examples = [
    {
    "question": "What do you know about France?",
    "answer":"""
    Here is what I know:
    Capital:Paris
    Language:French
    Food: Wine and Cheese
    Currency:Euro
    """,
    },
    {"question": "What do you know about Italy?",
     "answer": """
     I know this:
     Capital:Rome
     Language: Italian
     Food: Pizza and Pasta
     Curency: Euro
     """,
     },
    {"question": "What do you know about Greece?",
     "answer": """
     I know this:
     Capital: Athens
     Language:Greek
     Food: Souvlaki and Feta Cheese
     Currency:Euro
     """,
     },
]

In [3]:
# answer with Fewshot
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:

1. Capital: The capital city of France is Paris, which is also its largest city.

2. Language: The official language is French, and it is spoken by the majority of the population.

3. Geography: France shares borders with several countries, including Belgium, Luxembourg, Germany, Switzerland, Italy, Spain, and Andorra. It also has coastlines along the Mediterranean Sea, the Atlantic Ocean, and the English Channel.

4. History: France has a long and influential history, including the French Revolution in 1789, which led to the end of the monarchy and the rise of the First French Republic. It has also been involved in various conflicts, including World War I and World War II.

5. Culture: France is renowned for its cultural contributions, including its cuisine, fashion, art, literature, and cinema. It is ho

'France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:\n\n1. Capital: The capital city of France is Paris, which is also its largest city.\n\n2. Language: The official language is French, and it is spoken by the majority of the population.\n\n3. Geography: France shares borders with several countries, including Belgium, Luxembourg, Germany, Switzerland, Italy, Spain, and Andorra. It also has coastlines along the Mediterranean Sea, the Atlantic Ocean, and the English Channel.\n\n4. History: France has a long and influential history, including the French Revolution in 1789, which led to the end of the monarchy and the rise of the First French Republic. It has also been involved in various conflicts, including World War I and World War II.\n\n5. Culture: France is renowned for its cultural contributions, including its cuisine, fashion, art, literature, and cinem

In [4]:
# Need to format an example

#example_template = """
#    Human: {question}
#    AI: {answer}
#"""

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")

# now put it inside of the chain
chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI: 
     I know this:
     Capital: Berlin
     Language: German
     Food: Bratwurst and Sauerkraut
     Currency: Euro

AIMessageChunk(content='AI: \n     I know this:\n     Capital: Berlin\n     Language: German\n     Food: Bratwurst and Sauerkraut\n     Currency: Euro')

In [5]:
examples2 = [
    {
    "country": "France",
    "answer":"""
    Here is what I know:
    Capital:Paris
    Language:French
    Food: Wine and Cheese
    Currency:Euro
    """,
    },
    {"country": "Italy",
     "answer": """
     I know this:
     Capital:Rome
     Language: Italian
     Food: Pizza and Pasta
     Curency: Euro
     """,
     },
    {"country": "Greece",
     "answer": """
     I know this:
     Capital: Athens
     Language:Greek
     Food: Souvlaki and Feta Cheese
     Currency:Euro
     """,
     },
]

In [6]:
example2_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}? "),
    ("ai", "{answer}"),
])

example2_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example2_prompt,
    examples=examples2,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answer"),
    example2_prompt,
    ("human", "what do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke({"country":"Germany"})

I know this:
     Capital: Berlin
     Language: German
     Food: Bratwurst and Sauerkraut
     Currency: Euro

AIMessageChunk(content='I know this:\n     Capital: Berlin\n     Language: German\n     Food: Bratwurst and Sauerkraut\n     Currency: Euro')

### How to make a dynamic example selector
- this will format your example, see how long the are 
- depending on your setting, it is going to choose example that fit on the prompt

In [7]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

# examples from above cell
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n    Here is what I know:\n    Capital:Paris\n    Language:French\n    Food: Wine and Cheese\n    Currency:Euro\n    \n\nHuman: What do you know about Italy?\nAI: \n     I know this:\n     Capital:Rome\n     Language: Italian\n     Food: Pizza and Pasta\n     Curency: Euro\n     \n\nHuman: What do you know about Greece?\nAI: \n     I know this:\n     Capital: Athens\n     Language:Greek\n     Food: Souvlaki and Feta Cheese\n     Currency:Euro\n     \n\nHuman: What do you know about Brazil?'

In [8]:
from typing import Any, Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector


# this is a very rough simple example
# from list of example above, choose one randomly
class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] # select_examples should return a list!
    
    
    
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n     I know this:\n     Capital:Rome\n     Language: Italian\n     Food: Pizza and Pasta\n     Curency: Euro\n     \n\nHuman: What do you know about Brazil?'

### Composition
- How to have various tiny prompt template and put them all together

- How to load prompt templates from the disk

In [11]:
from langchain.prompts import load_prompt


prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

- This allow us to combine many prompts together.

In [12]:
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg! That is a secret!! Arg arg!!",
    "question":"What is your fav food?",
})


Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!")

## Caching
- https://python.langchain.com/docs/integrations/llms/llm_caching

In [18]:
# from langchain.chat_models import ChatOpenAI
# from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug

# cache we are currently using is in memory cache
# so, if we restart the notebook, it takes 17 sec again.
from langchain.cache import InMemoryCache, SQLiteCache # hence, keep a cache on a database (SQL)


set_llm_cache(InMemoryCache())
# debug is useful when working with chain
set_debug(True) # this command shows what's going on with prompt

chat = ChatOpenAI(
    temperature=0.1,
)

In [19]:
chat.predict("How do you make italian pasta") # this response took around 17 sec

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [12.66s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little mor

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough in

In [20]:
# This took immediately since it did not go through LLM but cache
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flou

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little bit at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough in

In [21]:
set_llm_cache(SQLiteCache("cache.db"))
chat.predict("How to study data structure?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to study data structure?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [12.48s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here are some steps to help you study data structures effectively:\n\n1. Understand the basics: Start by understanding the fundamental concepts of data structures, such as arrays, linked lists, stacks, queues, trees, graphs, and hash tables. Learn about their properties, operations, and use cases.\n\n2. Choose a learning resource: Select a reliable learning resource that suits your learning style. This could be a textbook, online course, video tutorials, or interactive coding platforms.\n\n3. Break it down: Divide your study into smaller topics and focus on one data structure at a time. This will help you grasp the concepts more easily and avoid feeling overwhelmed.\n\n4. Implement and code: Practice implementing data structures in your preferred

'Here are some steps to help you study data structures effectively:\n\n1. Understand the basics: Start by understanding the fundamental concepts of data structures, such as arrays, linked lists, stacks, queues, trees, graphs, and hash tables. Learn about their properties, operations, and use cases.\n\n2. Choose a learning resource: Select a reliable learning resource that suits your learning style. This could be a textbook, online course, video tutorials, or interactive coding platforms.\n\n3. Break it down: Divide your study into smaller topics and focus on one data structure at a time. This will help you grasp the concepts more easily and avoid feeling overwhelmed.\n\n4. Implement and code: Practice implementing data structures in your preferred programming language. Start with simple examples and gradually move on to more complex ones. This hands-on approach will solidify your understanding and improve your coding skills.\n\n5. Analyze time and space complexity: Understand the time 

In [22]:
chat.predict("How to study data structure?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How to study data structure?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here are some steps to help you study data structures effectively:\n\n1. Understand the basics: Start by understanding the fundamental concepts of data structures, such as arrays, linked lists, stacks, queues, trees, graphs, and hash tables. Learn about their properties, operations, and use cases.\n\n2. Choose a learning resource: Select a reliable learning resource that suits your learning style. This could be a textbook, online course, video tutorials, or interactive coding platforms.\n\n3. Break it down: Divide your study into smaller topics and focus on one data structure at a time. This will help you grasp the concepts more easily and avoid feeling overwhelmed.\n\n4. Implement and code: Practice implementing data structures in your preferred progr

'Here are some steps to help you study data structures effectively:\n\n1. Understand the basics: Start by understanding the fundamental concepts of data structures, such as arrays, linked lists, stacks, queues, trees, graphs, and hash tables. Learn about their properties, operations, and use cases.\n\n2. Choose a learning resource: Select a reliable learning resource that suits your learning style. This could be a textbook, online course, video tutorials, or interactive coding platforms.\n\n3. Break it down: Divide your study into smaller topics and focus on one data structure at a time. This will help you grasp the concepts more easily and avoid feeling overwhelmed.\n\n4. Implement and code: Practice implementing data structures in your preferred programming language. Start with simple examples and gradually move on to more complex ones. This hands-on approach will solidify your understanding and improve your coding skills.\n\n5. Analyze time and space complexity: Understand the time 

### Serialization
- Saving and loading models

- What if you want to know how much money cost on each call?

In [24]:
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

# for 'usage', you can name it whatever you want
with get_openai_callback() as usage:
    #call your llm
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a simple recipe for making soju at home:\n\nIngredients:\n- 1 cup of rice\n- 1 cup of nuruk (a Korean fermentation starter)\n- 8 cups of water\n- 1 tablespoon of yeast\n- 1 tablespoon of sugar (optional)\n\nInstructions:\n1. Rinse the rice thoroughly until the water runs clear.\n2. Soak the rice in water for about 1 hour, then drain.\n3. Steam the rice until it becomes soft and sticky.\n4. Transfer the steamed rice to a large container or pot.\n5. Add nuruk to the rice and mix well.\n6. Boil 8 cups of water and let it cool down to room temperature.\n7. Add the cooled water to the rice and nuruk mixture.\n8. Dissolve yeast in a small amount of warm water and add it to the mixture.\n9. Stir everything together until well combined.\n10. Cover the c

## Serialization

- Create your own model and save it

In [26]:
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model='gpt-3.5-turbo-16k'
)

chat.save("gpt3.5_model.json")

- You can also load your model

In [27]:
from langchain.llms.loading import load_llm

chat = load_llm("gpt3.5_model.json")
chat

/Users/joongho/Documents/fullstack_gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/joongho/Documents/fullstack_gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})